# T2.2: PANDAS. Ler excels de alcaldes



<p>Instalación de paquetes necesarios</p>

In [ ]:
%conda install pandas # o  %pip install pandas
%conda install datetime #o %pip install datetime
%conda install unidecode # o %pip install unidecode

<p>Importar librerías</p>

In [1]:
import pandas as pd
from datetime import datetime
from unidecode import unidecode


### Normalización del data frame
<p>Cargo todas las páginas del excel y creo el método para limpiar y formatear los datos, después volvemos a juntar las hojas limpiadas.</p>

In [2]:
excel_entero = pd.read_excel("Alcades_1979_2023.xlsx", na_values='\\N', sheet_name=None)

def limpiar(df): 
    # Quitar los valores NaN
    df = df.dropna(axis=1, how='all') 

    # Quitar las columnas innecesarias
    df = df.iloc[7:]

    # Quitar una columna vacía
    df = df.drop(['Unnamed: 13'], axis=1) 

    # Renombrar  los header
    df.rename(columns={'Unnamed: 3': 'COMUNIDAD AUTONOMA', 'Unnamed: 4': 'PROVINCIA', 'Unnamed: 5': 'MUNICIPIO', 'Unnamed: 6': 'CODIGO INE', 'Unnamed: 7': 'NOMBRE', 'Unnamed: 8': 'LISTA', 'Unnamed: 9': 'FECHA POSESION', 'Unnamed: 10': 'FECHA BAJA'}, inplace=True)
    

    # Eliminar las filas donde el nombre este vacío
    df = df[df['NOMBRE'].notna() & (df['NOMBRE'] != '')]


    # Cambiar el formato de la fecha 
    for fecha_col in ['FECHA POSESION', 'FECHA BAJA']: 
        if fecha_col in df.columns:
            df[fecha_col] = pd.to_datetime(df[fecha_col], errors='coerce', dayfirst=True, format="%d/%m/%Y").dt.strftime('%d/%m/%Y')

    # Quitar los  acentos y convertirlo a minuscula
    df = df.map(lambda texto: unidecode(
        texto.lower().strip()
        .replace(',', '')
        .replace('.', '')
        .replace('*', '')
        .replace('-', ' ')) 
        if isinstance(texto, str) else texto)

    return df 

hojas_limpiadas = []

# Limpiar las hojas y concatenarlas en un solo Dataframe
for hoja in excel_entero:
    hoja_limpiada = limpiar(excel_entero[hoja])  
    hojas_limpiadas.append(hoja_limpiada)  
    

excel_entero_df = pd.concat(hojas_limpiadas, ignore_index=True)



excel_entero_df.head(50)




,COMUNIDAD AUTONOMA,PROVINCIA,MUNICIPIO,CODIGO INE,NOMBRE,LISTA,FECHA POSESION,FECHA BAJA
0,andalucia,almeria,abla,040010,herrerias herrerias antonio,ucd,19/04/1979,NaN
1,andalucia,almeria,abrucena,040025,martinez lao juan,psoe,19/04/1979,NaN
2,andalucia,almeria,adra,040031,sarmiento posada pedro,otros,19/04/1979,NaN
3,andalucia,almeria,albanchez,040046,martos aybar ramon,ucd,19/04/1979,NaN
4,andalucia,almeria,alboloduy,040059,blanes paniagua mariano,ucd,19/04/1979,NaN
5,andalucia,almeria,albox,040062,miras carrasco jose,ucd,19/04/1979,NaN
6,andalucia,almeria,alcolea,040078,lopez mellado gabriel bernardo,ucd,19/04/1979,NaN
7,andalucia,almeria,alcontar,040084,requena martinez manuel,ucd,19/04/1979,NaN
8,andalucia,almeria,alcudia de monteagud,040097,perez padilla rogelio,ucd,19/04/1979,NaN
9,andalucia,almeria,alhabia,040101,castellon diaz francisco,ucd,19/04/1979,NaN


### Fai algunhas averiguacións sobre os datos:

<p>1. Saber qué alcaldes repetiron mandato e en qué lexislaturas (ollo, poden mudar un pouco os nomes).</p>

In [3]:
import pandas as pd

# Agrupar por nombre y municipio, y contar las legislaturas
repetidos = (
    excel_entero_df.groupby(['NOMBRE', 'MUNICIPIO'])
    .size()
    .reset_index(name='FRECUENCIA')
)

# Filtrar alcaldes que repitieron mandato 
alcaldes_repetidos = repetidos[repetidos['FRECUENCIA'] > 1]

# Filtrar el dataframe original para incluir solo alcaldes que repitieron mandato
df_repetidos = excel_entero_df.merge(alcaldes_repetidos, on=['NOMBRE', 'MUNICIPIO'])

# Convertir las fechas de posesión a formato datetime y extraer solo los años
df_repetidos['FECHA POSESION'] = pd.to_datetime(df_repetidos['FECHA POSESION'], errors='coerce', dayfirst=True)
df_repetidos = df_repetidos.dropna(subset=['FECHA POSESION'])  # Eliminar fechas inválidas

# Agrupar por nombre y municipio, y extraer las legislaturas (años) únicos en los que participaron
legislatura_por_alcalde = (
    df_repetidos.groupby(['NOMBRE', 'MUNICIPIO'])['FECHA POSESION']
    .apply(lambda x: sorted(x.dt.year.unique()))
    .reset_index(name='LEGISLATURAS')
)

# Filtrar solo los alcaldes que participaron en más de una legislatura
legislatura_por_alcalde = legislatura_por_alcalde[legislatura_por_alcalde['LEGISLATURAS'].apply(len) > 1]

# Mostrar resultados
legislatura_por_alcalde


,NOMBRE,MUNICIPIO,LEGISLATURAS
0,abad becquer fernando,leganes,"[1983, 1987]"
1,abad benedicto enrique,rinconada (la),"[1987, 1991, 1995, 1999]"
2,abad caballero jose,carabantes,"[1983, 1987]"
3,abad garcia antonio,santa cruz de marchena,"[1991, 1995, 1999]"
4,abad gomez heradio,poblacion de campos,"[1979, 1983, 1991]"
...,...,...,...
21390,zuniga perez lemaur jesus,rozas de madrid (las),"[1983, 1987, 1991]"
21391,zurbano lecumberri jose,cabredo,"[1979, 1983, 1991, 1997]"
21392,zuriaga adrian blas,yesa (la),"[1979, 1991]"
21393,zurriaga manez vicente,olocau,"[1979, 1983]"


<p>2. De repetir mandato. Qué partidos políticos tiveron máis éxito? Qué alcaldes mudaron de partido?</p>

In [5]:
# Contar cuántas veces cada partido tuvo alcaldes reelectos
partidos_exito = df_repetidos.groupby('LISTA').size().reset_index(name='ALCALDES REELECTOS')

# Ordenar de mayor a menor
partidos_exito = partidos_exito.sort_values(by='ALCALDES REELECTOS', ascending=False)

print("Partidos más exitosos: ")
print(partidos_exito)


# Agrupar por alcalde y municipio para obtener los partidos únicos en los que ha estado
cambios_partido = (
    df_repetidos.groupby(['NOMBRE', 'MUNICIPIO'])['LISTA']
    .apply(lambda x: sorted(x.unique()))
    .reset_index(name='LISTA')
)

# Filtrar solo los alcaldes que han cambiado de partido.
cambios_partido = cambios_partido[cambios_partido['LISTA'].apply(len) > 1]


print("Cambios de partido: ")
# Mostrar resultados
cambios_partido



Partidos más exitosos: 
      LISTA  ALCALDES REELECTOS
71     psoe               19553
62       pp               18729
47      ind                3325
25      ciu                2962
51    otros                1818
..      ...                 ...
69      psg                   1
30       cs                   1
61  podemos                   1
43  ganemos                   1
34       ee                   1

[89 rows x 2 columns]
Cambios de partido: 


,NOMBRE,MUNICIPIO,LISTA
2,abad caballero jose,carabantes,"[ap, ap/pdp/ul]"
4,abad gomez heradio,poblacion de campos,"[ap/pdp/ul, pp, ucd]"
5,abad pastor eugenio,cabra de mora,"[ap/pdp/ul, ind]"
7,abad roldan amadeo,congosto de valdavia,"[pp, ucd]"
9,abadia urieta armando,jaca,"[ap, ap/pdp/ul, pp, ucd]"
...,...,...,...
21386,zubizarreta lasagabaster xabier,arrasate o mondragon,"[eh, hb]"
21388,zunbeltz bedialauneta ibaibarriaga,ondarroa,"[bildu, bildu ea]"
21391,zurbano lecumberri jose,cabredo,"[cgest, ind, upn]"
21392,zuriaga adrian blas,yesa (la),"[pp, ucd]"


<p>3. Cantos alcaldes cesaron antes de tempo?</p>

In [7]:
# Convertir las fechas a formato datetime
excel_entero_df['FECHA POSESION'] = pd.to_datetime(excel_entero_df['FECHA POSESION'], errors='coerce',dayfirst=True)
excel_entero_df['FECHA BAJA'] = pd.to_datetime(excel_entero_df['FECHA BAJA'], errors='coerce',dayfirst=True)

# Calcular la duración del mandato en años
excel_entero_df['DURACION'] = (excel_entero_df['FECHA BAJA'] - excel_entero_df['FECHA POSESION']).dt.days / 365

# Filtrar los  alcaldes que cesaron antes de 4 años
ceses_anticipados = excel_entero_df[excel_entero_df['DURACION'] < 4]


total_cesados = ceses_anticipados.shape[0]

print(f"Total de alcaldes que cesaron antes de tiempo: {total_cesados}")



Total de alcaldes que cesaron antes de tiempo: 6476


<p>4. Cal é o soldo máis alto e de que alcaldes?</p>

In [27]:
ultima_hoja_df = limpiar(excel_entero["Alcaldes 2019-2023"])

retribuciones2018_df = pd.read_csv("tabula-retribuciones_alcaldes_2021.csv")

# Convertir la columna 'TOTAL PERCIBIDO' a float para poder obtener el sueldo de los alcaldes
retribuciones2018_df["TOTAL PERCIBIDO"] = (
    retribuciones2018_df["TOTAL PERCIBIDO"]
    .str.replace(".", "", regex=False) 
    .str.replace(",", ".", regex=False) 
    .astype(float)
)

# Normalizar las columnas
retribuciones2018_df[["AYUNTAMIENTO", "PROVINCIA"]] = retribuciones2018_df[["AYUNTAMIENTO", "PROVINCIA"]].map(lambda x: unidecode(x.lower().strip()) if isinstance(x, str) else x)


# Hacer el merge con los nombres normalizados
df_merged = ultima_hoja_df.merge(
    retribuciones2018_df,
    left_on=["MUNICIPIO", "PROVINCIA"],
    right_on=["AYUNTAMIENTO", "PROVINCIA"]
    )

# Seleccionar columnas necesarias
df_ordenado = df_merged[["NOMBRE", "MUNICIPIO", "PROVINCIA", "TOTAL PERCIBIDO"]]

# Ordenar por sueldo de mayor a menor
df_ordenado = df_ordenado.sort_values(by="TOTAL PERCIBIDO", ascending=False)

print("Top 10 alcaldes con mayor sueldo: ")
df_ordenado.head(10)


Top 10 alcaldes con mayor sueldo: 


,NOMBRE,MUNICIPIO,PROVINCIA,TOTAL PERCIBIDO
5824,jose luis martinez almeida navasques,madrid,madrid,108517.80
4133,ada colau ballano,barcelona,barcelona,100000.04
6302,juan maria aburto rike,bilbao,bizkaia,98018.40
634,juan espadas cejas,sevilla,sevilla,88673.68
635,antonio munoz martinez,sevilla,sevilla,88673.68
3215,oscar puente santiago,valladolid,valladolid,87595.62
5253,joan ribo canut,valencia,valencia/valencia,87137.82
6323,amaia agirre munoa,getxo,bizkaia,83985.57
12,ramon fernandez pacheco monterreal,almeria,almeria,83376.86
13,maria del mar vazquez aguero,almeria,almeria,83376.86


<p>5. Relaciona os soldos altos e baixos coa "probabilidade" de repetir mandato.</p>

In [33]:
# Dire que un sueldo bajo es por debajo del SMI ANUAL actualmente: 15876

"""retribuciones2018_dfalto =  

retribuciones2018_dfalto"""

retribuciones2018_df['TOTAL PERCIBIDO'] 

df_merged = ultima_hoja_df.merge(
    retribuciones2018_df,
    left_on=["MUNICIPIO", "PROVINCIA"],
    right_on=["AYUNTAMIENTO", "PROVINCIA"]
    )

df_merged


,COMUNIDAD AUTONOMA,PROVINCIA,MUNICIPIO,CODIGO INE,NOMBRE,LISTA,FECHA POSESION,FECHA BAJA,AYUNTAMIENTO,CCAA,RÉGIMEN DEDICACIÓN,TOTAL PERCIBIDO
0,andalucia,almeria,abla,040010,antonio fernandez lopez,pp,15/06/2019,NaN,abla,Andalucía,Exclusiva,30356.62
1,andalucia,almeria,abrucena,040025,ismael gil salmeron,psoe,15/06/2019,NaN,abrucena,Andalucía,Exclusiva,34270.18
2,andalucia,almeria,adra,040031,manuel cortes perez,pp,15/06/2019,NaN,adra,Andalucía,Exclusiva,57584.20
3,andalucia,almeria,albanchez,040046,amador lopez pardo,c's,15/06/2019,NaN,albanchez,Andalucía,Exclusiva,23572.48
4,andalucia,almeria,alboloduy,040059,sonia maria guil soriano,pp,15/06/2019,NaN,alboloduy,Andalucía,Parcial,24300.04
...,...,...,...,...,...,...,...,...,...,...,...,...
6433,pais vasco,gipuzkoa,zumarraga,200809,mikel serrano aperribay,psoe,15/06/2019,NaN,zumarraga,País Vasco,Exclusiva,65697.80
6434,pais vasco,gipuzkoa,baliarrain,209040,jaione olano etxeberria,otros,19/05/2020,21/03/2021,baliarrain,País Vasco,Sin dedicación,0.00
6435,pais vasco,gipuzkoa,baliarrain,209040,juan ma iturrioz goni,otros,21/03/2021,13/03/2022,baliarrain,País Vasco,Sin dedicación,0.00
6436,pais vasco,gipuzkoa,baliarrain,209040,andoni olano manzano,otros,13/03/2022,15/11/2022,baliarrain,País Vasco,Sin dedicación,0.00


<p>6. Outra información interesante.</p>
